## Exploring Blooms Labs DMS Tools package

Using Bloom Lab DMS Tool's findSigSel function (https://jbloomlab.github.io/dms_tools2/dms_tools2.plot.html#dms_tools2.plot.findSigSel) on Abby Clyde's DMS data

In [ ]:
import re
import os
import math
import numbers
import random
import collections

# import natsort
import pandas
import numpy
import scipy.stats
import scipy.optimize
from statsmodels.sandbox.stats.multicomp import multipletests

import matplotlib.pyplot as plt
import seaborn as sns

COLOR_BLIND_PALETTE = ["#000000", "#E69F00", "#56B4E9", "#009E73",
                       "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]

## Function: findSigSel

Following example vignette from https://jbloomlab.github.io/dms_tools2/dms_tools2.plot.html#dms_tools2.plot.findSigSel

Manually adding source code

In [ ]:
def findSigSel(df, valcol, plotfile, fdr=0.05, title=None,
               method='robust_hist', mle_frac_censor=0.005,
               returnplot=False):
    """Finds "significant" selection at sites / mutations.

    Designed for the case where most sites / mutations are not
    under selection, but a few may be. It tries to find those
    few that are under selection. 

    It does not use a mechanistic statistical model, but rather fits
    a gamma distribution. The rationale for
    a gamma distribution is that it is negative binomial's continuous
    `analog <http://www.nehalemlabs.net/prototype/blog/2013/12/01/gamma-distribution-approximation-to-the-negative-binomial-distribution/>`_.
    It then identifies sites that clearly have **larger** values than
    expected under this distribution. It currently does not
    identify sites with **smaller** (or more negative) than expected
    values.

    Args:
        `df` (pandas DataFrame)
            Contains data to analyze
        `valcol` (string)
            Column in `df` with values (e.g., `fracsurvive`)
        `plotfile` (string)
            Name of file to which we plot fit.
        `fdr` (float)
            Find sites that are significant at this `fdr`
            given fitted distribution.
        `title` (string or `None`)
            Title for plot.
        `method` (str)
            Specifies how to fit gamma distribution, can have following values:

              - 'robust_hist': bin the data, then use 
                `robust regression <http://scipy-cookbook.readthedocs.io/items/robust_regression.html>`_
                (soft L1 loss) to fit a gamma distribution to the histogram.

              - 'mle': fit the gamma distribution to the points by maximum
                likelihood; see also `mle_frac_censor`.

        `mle_frac_censor` (float)
            Only meaningful if `method` is 'mle'. In this case, before fitting,
            censor the data by setting the top `mle_frac_censor` largest values
            to the `mle_frac_censor` largest value. This shrinks very large
            outliers that affect fit.
        `returnplot` (bool)
            Return the matplotlib figure.

    Returns:
        Creates the plot in `plotfile`. Also returns
        the 3-tuple `(df_sigsel, cutoff, gamma_fit)` where:

            - `df_sigsel` is copy of `df` with new columns
              `P`, `Q`, and `sig`. These give P value, Q
              value, and whether site meets `fdr` cutoff
              for significance.

            - `cutoff` is the maximum value that is **not**
              called significant. Because FDR is a property
              of a distribution, this value cannot be 
              interpreted as meaning a new data point would
              be called based on this cutoff, as the cutoff
              would change. But `cutoff` is useful for 
              plotting to see significant / non-significant.

            - `gamma_params` is a `numpy.ndarray` that of 
              length 3 that gives the shape, scale, and location
              parameter of the fit gamma distribution.

        If `returnplot` is `True`, then return
        `((df_sigsel, cutoff, gamma_fit), fig)` where `fig`
        is the matplotlib figure.

    An example: First, simulate points from a gamma distribution:

    .. nbplot::

        >>> import pandas
        >>> import scipy
        >>> from dms_tools2.plot import findSigSel
        >>>
        >>> shape_sim = 1.5
        >>> scale_sim = 0.005
        >>> loc_sim = 0.0
        >>> gamma_sim = scipy.stats.gamma(shape_sim, scale=scale_sim,
        ...         loc=loc_sim)
        >>> nsites = 1000
        >>> scipy.random.seed(0)
        >>> df = pandas.DataFrame.from_dict({
        ...         'site':[r for r in range(nsites)],
        ...         'fracsurvive':gamma_sim.rvs(nsites)})

    Now make two sites have "significantly" higher values:

    .. nbplot::

        >>> sigsites = [100, 200]
        >>> df.loc[sigsites, 'fracsurvive'] = 0.08

    Now find the significant sites:

    .. nbplot::

        >>> plotfile = '_findSigSel.png'
        >>> (df_sigsel, cutoff, gamma_params) = findSigSel(
        ...         df, 'fracsurvive', plotfile, title='example')

    Make sure the fitted params are close to the ones used to
    simulate the data:

    .. nbplot::

        >>> numpy.allclose(shape_sim, gamma_params[0], rtol=0.1, atol=1e-3)
        True
        >>> numpy.allclose(scale_sim, gamma_params[1], rtol=0.1, atol=1e-3)
        True
        >>> numpy.allclose(loc_sim, gamma_params[2], rtol=0.1, atol=1e-3)
        True

    Check that we find the correct significant sites:

    .. nbplot::

        >>> set(sigsites) == set(df_sigsel.query('sig').site)
        True

    Make sure that sites above cutoff are significant:

    .. nbplot::

        >>> df_sigsel.query('sig').equals(df_sigsel.query('fracsurvive > @cutoff'))
        True

    Now repeat, getting and showing the plot:

    .. nbplot::

        >>> _, fig = findSigSel(
        ...         df, 'fracsurvive', plotfile, title='example', returnplot=True)

    Now use the 'mle' `method`:

    .. nbplot::

        >>> (df_sigsel_mle, _, _), fig_mle = findSigSel(
        ...         df, 'fracsurvive', plotfile, title='example',
        ...         method='mle', returnplot=True)
        >>> set(df_sigsel_mle.query('sig').site) == set(sigsites)
        True

    """
    assert valcol in df.columns, "no `valcol` {0}".format(valcol)

    newcols = {'P', 'Q', 'sig'}
    assert not (newcols & set(df.columns)), \
            "`df` already has {0}".format(newcols)

    # We fit curves to histogram. First we need to get bins.
    try:
        # try with Freedman Diaconis Estimator
        binedges = numpy.histogram(df[valcol], bins='fd')[1]
    except ValueError:
        # fd will fail of lots of identical points
        binedges = numpy.histogram(df[valcol], bins='doane')[1]

    # get bin centers
    bins = (binedges[ : -1] + binedges[1 : ]) / 2

    # plot the histogram
    plt.figure(figsize=(5.5, 4))
    (heights, binedges, patches) = plt.hist(df[valcol],
            bins=binedges, density=True, histtype='stepfilled',
            color=COLOR_BLIND_PALETTE[2])

    # initial guess gives correct mean and variance for
    # gamma distribution with loc of 0
    scale = df[valcol].var() / df[valcol].mean()
    shape = df[valcol].mean() / scale
    x0 = numpy.array([shape, scale, 0.0])

    if method == 'robust_hist':
        def _f(x, bins, heights):
            """Gamma distribution least squares fitting function.

            Zero when distribution perfectly fits histogram.
            `x` is `(shape, scale, loc)`.
            """
            return (scipy.stats.gamma.pdf(bins, x[0], scale=x[1],
                    loc=x[2]) - heights)

        # fit using soft L1 loss for robust regression
        # http://scipy-cookbook.readthedocs.io/items/robust_regression.html
        fit = scipy.optimize.least_squares(_f, x0, args=(bins, heights),
                                           loss='soft_l1')
        gamma_params = fit.x
        gamma_fit = scipy.stats.gamma(fit.x[0], scale=fit.x[1],
                                      loc=fit.x[2])

    elif method == 'mle':
        vals = numpy.sort(df[valcol].values)
        mle_n_censor = round(len(vals) * mle_frac_censor)
        maxval = vals[-1 - mle_n_censor]
        vals[vals > maxval] = maxval
        fit_shape, fit_loc, fit_scale = scipy.stats.gamma.fit(vals,
                                                              shape,
                                                              scale=scale,
                                                              loc=0)
        gamma_params = numpy.array([fit_shape, fit_scale, fit_loc])
        gamma_fit = scipy.stats.gamma(fit_shape, scale=fit_scale, loc=fit_loc)

    else:
        raise ValueError(f"invalid `method` {method}")

    # add fit gamma distribution to plot
    nfitbins = 500
    if nfitbins > len(bins):
        fitbins = numpy.linspace(bins[0], bins[-1], nfitbins)
    else:
        fitbins = bins
    plt.plot(fitbins, gamma_fit.pdf(fitbins),
            color=COLOR_BLIND_PALETTE[1])

    # compute P and Q values
    df_sigsel = (df.assign(P=lambda x: gamma_fit.sf(x[valcol]))
                   .assign(Q=lambda x: multipletests(x.P, fdr, 'fdr_bh')[1])
                   .assign(sig=lambda x: multipletests(x.P, fdr, 'fdr_bh')[0])
                   )

    # compute cutoff 
    cutoff = df_sigsel.query('not sig')[valcol].max()

    # plot cutoff
    # find first bin boundary greater than cutoff
    if (binedges > cutoff).any():
        bincutoff = binedges[binedges > cutoff][0]
    else:
        bincutoff = binedges[-1]
    # now annotate plot
    plt.axvline(bincutoff, color=COLOR_BLIND_PALETTE[3], ls='--', lw=0.75)
    text_y = 0.95 * plt.ylim()[1]
    (xmin, xmax) = plt.xlim()
    if (bincutoff - xmin) < 0.75 * (xmax - xmin):
        text_x = bincutoff + 0.01 * (xmax - xmin)
        ha = 'left'
    else:
        text_x = bincutoff - 0.01 * (xmax - xmin)
        ha = 'right'
    if len(df_sigsel.query('sig')):
        text = '{0} values\nsignificant\n($>${1})'.format(
                len(df_sigsel.query('sig')), latexSciNot([cutoff])[0])
    else:
        text = 'no values\nsignificant'
    plt.text(text_x, text_y, text,
            horizontalalignment=ha, verticalalignment='top',
            color=COLOR_BLIND_PALETTE[3], size='small')

    # put labels on plot
    plt.xlabel(valcol.replace('_', ' '))
    plt.ylabel('density')
    if title:
        plt.title(title.replace('_', ' '))

    # save plot
    plt.tight_layout()
    plt.savefig(plotfile)

    if returnplot:
        return ((df_sigsel, cutoff, gamma_params), plt.gcf())
    else:
        plt.close()
        return (df_sigsel, cutoff, gamma_params)

    
def latexSciNot(xlist):
    """
    Converts list of numbers to LaTex scientific notation.
    """
    if isinstance(xlist, numbers.Number):
        isnum = True
        xlist = [xlist]
    else:
        isnum = False
    formatlist = []
    for x in xlist:
        xf = "{0:.2g}".format(x)
        if xf[ : 2] == '1e':
            xf = "$10^{{{0}}}$".format(int(xf[2 : ]))
        elif xf[ : 3] == '-1e':
            xf = "$-10^{{{0}}}$".format(int(xf[3 : ]))
        elif 'e' in xf:
            (d, exp) = xf.split('e')
            xf = '${0} \\times 10^{{{1}}}$'.format(d, int(exp))
        else:
            xf = '${0}$'.format(xf)
        formatlist.append(xf)
    if isnum:
        assert len(formatlist) == 1
        formatlist = formatlist[0]
    return formatlist

In [ ]:
# simulate gamme distributed data
shape_sim = 1.5
scale_sim = 0.005
loc_sim = 0.0
gamma_sim = scipy.stats.gamma(shape_sim, scale=scale_sim, loc=loc_sim)
nsites = 1000 # 1000 sites
#scipy.random.seed(0)
df = pandas.DataFrame.from_dict({
    'site':[r for r in range(nsites)],
    'fracsurvive':gamma_sim.rvs(nsites)})

In [ ]:
# create two points to stand out
sigsites = [100, 200] # significant sites
df.loc[sigsites, 'fracsurvive'] = 0.08 # setting frac survival to 8% for site at 100 and  at 200

In [ ]:
# create an example file to ensure function works
plotfile = '_findSigSel.png' # file to print plot to
(df_sigsel, cutoff, gamma_params) = findSigSel(df, 'fracsurvive', plotfile, title='example')

- `df_sigsel` is copy of `df` with new columns
  `P`, `Q`, and `sig`. These give P value, Q
  value, and whether site meets `fdr` cutoff
  for significance.

- `cutoff` is the maximum value that is **not**
  called significant. Because FDR is a property
  of a distribution, this value cannot be 
  interpreted as meaning a new data point would
  be called based on this cutoff, as the cutoff
  would change. But `cutoff` is useful for 
  plotting to see significant / non-significant.

- `gamma_params` is a `numpy.ndarray` that of 
  length 3 that gives the shape, scale, and location
  parameter of the fit gamma distribution.

In [ ]:
# viewing gamma params
gamma_params

## Using bNAb data files 

`data` files: BF520_10-1074_mut_effect, BF520_3BNC117_mut_effect, TRO11_10-1074_mut_effect, TRO11_3BNC117_mut_effect

In [ ]:
file = 'BF520_3BNC117' # change filename here
path = 'data/' + file + '_mut_effect.csv'
df = pandas.read_csv(path)
df.shape

In [ ]:
# Filter for times_seen > 3

df = df[df['times_seen']>3]
#df = df[df['escape_median'] >= 0]

The positive versus negative values for escape are different: positive values represent escape while negative values represent higher neutralization. So, we can fit two separate gamma distributions; one for escape effects and one for sensitization effects. 

You would just take the absolute value of the mutations that cause sensitization and use those so that they are positive rather than negative for the gamma distribution fitting.

In [ ]:
# Separating sensitization and neutralization effects

# if using Escape Median instead of Mean

# df_neu = df[df['escape_median'] > 0]
# df_sen = df[df['escape_median'] < 0]

# df_sen['escape_median'] = numpy.abs(df_sen['escape_median'])

# using Escape Mean instead of Median
df_neu = df[df['escape_mean'] > 0]
df_sen = df[df['escape_mean'] < 0]

df_sen['escape_mean'] = numpy.abs(df_sen['escape_mean'])

In [ ]:
# # FOR MEDIAN
# for i in ['neutralization', 'sensitization']:
#     if i == 'neutralization':
#         df = df_neu
#         plt_name = file + '_Neutralization'
#     else:
#         df = df_sen
#         plt_name = file + '_Sensitization'
#     plotfile = f'{plt_name}_findSigSel.png' # file to print plot to
#     (df_sigsel, cutoff, gamma_params) = findSigSel(df, 'escape_median', plotfile, title=plt_name)
    
#     df_sigsel[df_sigsel['sig'] != False].to_csv(f'{plt_name}.csv', index=False)

In [ ]:
# FOR MEAN
for i in ['neutralization', 'sensitization']:
    if i == 'neutralization':
        df = df_neu
        plt_name = file + '_Neutralization_Mean'
    else:
        df = df_sen
        plt_name = file + '_Sensitization_Mean'
    plotfile = f'{plt_name}_findSigSel.png' # path to print plot 
    (df_sigsel, cutoff, gamma_params) = findSigSel(df, 'escape_mean', plotfile, title=plt_name)
    # save the output as a CSV
    df_sigsel[df_sigsel['sig'] != False].to_csv(f'{plt_name}.csv', index=False)